In [2]:
import pandas as pd 
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import LinearRegression
import keras 
from keras.models import Sequential
from keras.layers import Dense

In [6]:
def cash_echannel(cash_echannel_yesterday,cash_in_echannel_today,cash_out_echannel_today):
    return cash_echannel_yesterday + cash_in_echannel_today + cash_out_echannel_today
def cash_office_today(cash_office_yesterday,cash_in_office_today,cash_out_office_today) : 
    return cash_office_yesterday + cash_in_office_today + cash_out_office_today

In [7]:
data_desc = pd.read_csv('data_description.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
predict_len = 31
test = test.iloc[0:31]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\microsoft user\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-fb9c95edea96>", line 1, in <module>
    data_desc = pd.read_csv('data_description.csv')
  File "c:\users\microsoft user\appdata\local\programs\python\python37\lib\site-packages\pandas\io\parsers.py", line 605, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "c:\users\microsoft user\appdata\local\programs\python\python37\lib\site-packages\pandas\io\parsers.py", line 457, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "c:\users\microsoft user\appdata\local\programs\python\python37\lib\site-packages\pandas\io\parsers.py", line 814, in __init__
    self._engine = self._make_engine(self.engine)
  File "c:\users\microsoft user\appdata\local\programs\python\python37\lib\site-packages\pandas\io\pars

FileNotFoundError: [Errno 2] No such file or directory: 'data_description.csv'

In [ ]:
print(train.shape)
print(test.shape)

## Prediksi Kas Kantor 

In [ ]:
x = np.array(train.index).reshape((-1, 1))
y = np.array(train.kas_kantor).reshape((-1,1))
model = LinearRegression()
model.fit(x, y)
r_sq = model.score(x, y)

plt.figure(figsize = (16,4))
plt.plot(model.predict(x))
plt.plot(np.array(train.kas_kantor).reshape((-1,1)))

x_test = np.array(range(425,425+predict_len)).reshape(-1,1)
x_test.shape
                        
y_pred_reg = model.predict(x_test)

plt.plot(x,y)
plt.plot(x_test,y_pred_reg)

In [ ]:
train_data_in = train.cash_in_echannel.iloc[:-len(test)].values
test_data_in = train.cash_in_echannel.iloc[-len(test):].values
train_data_out = train.cash_out_echannel.iloc[:-len(test)].values
test_data_out = train.cash_out_echannel.iloc[-len(test):].values

epochs = 500
batch_size = 50
verbose = 1
hidden_neuron1 = 31
hidden_neuron2 = 31
hidden_neuron3 = 31

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
raw_seq_in = train_data_in
# choose a number of time steps
n_steps_in, n_steps_out = len(test_data_in), len(test_data_in)
# split into samples
X_in, y_in = split_sequence(raw_seq_in, n_steps_in, n_steps_out)
# define model
model_in = Sequential()
model_in.add(Dense(hidden_neuron1, activation='relu', input_dim=n_steps_in))
model_in.add(Dense(hidden_neuron2, activation='relu', input_dim=n_steps_in))
model_in.add(Dense(hidden_neuron3, activation='relu', input_dim=n_steps_in))
model_in.add(Dense(n_steps_out))
model_in.compile(loss='mse', optimizer='adam')
es = keras.callbacks.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min',  restore_best_weights=True)
# fit model
history = model_in.fit(X_in, y_in, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=[es])

In [ ]:
# PLOT TRAIN and Validation LOSS
sns.set_style('white')
sns.set_context("paper", font_scale=1.4)  
plt.figure(figsize=(16,4))
plt.plot(history.history['loss'], color='r', linewidth=1, marker='d', markersize=7, label='Train Loss')
plt.plot(history.history['val_loss'], color='b',linewidth=1, marker='*',markersize=6, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation loss')
legend = plt.legend(loc='upper right',prop={'size': 12})

In [ ]:
# demonstrate prediction
x_input = y_in
# x_input = x_input.reshape((1, n_steps_in))
yhat_in = model_in.predict(x_input, verbose=0)
plt.figure(figsize = (16,4))
plt.plot(yhat_in[0])
plt.plot(test_data_in)

In [ ]:
# split a univariate sequence into samples

# define input sequence
raw_seq_out = train_data_out
# choose a number of time steps
n_steps_in, n_steps_out = len(test_data_out), len(test_data_out)
# split into samples
X_out, y_out = split_sequence(raw_seq_out, n_steps_in, n_steps_out)
# define model
model_out = Sequential()
model_out.add(Dense(hidden_neuron1, activation='relu', input_dim=n_steps_in))
model_out.add(Dense(hidden_neuron2, activation='relu', input_dim=n_steps_in))
model_out.add(Dense(hidden_neuron3, activation='relu', input_dim=n_steps_in))
model_out.add(Dense(n_steps_out))
model_out.compile(loss='mse', optimizer='adam')
es = keras.callbacks.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min',  restore_best_weights=True)
# fit model
history = model_out.fit(X_out, y_out, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=[es])

In [ ]:
# PLOT TRAIN and Validation LOSS
sns.set_style('white')
sns.set_context("paper", font_scale=1.4)  
plt.figure(figsize=(16,4))
plt.plot(history.history['loss'], color='r', linewidth=1, marker='d', markersize=7, label='Train Loss')
plt.plot(history.history['val_loss'], color='b',linewidth=1, marker='*',markersize=6, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation loss')
legend = plt.legend(loc='upper right',prop={'size': 12})

In [ ]:
# demonstrate prediction
x_input = y_out
# x_input = x_input.reshape((1, n_steps_in))
yhat_out = model_out.predict(x_input, verbose=0)
plt.figure(figsize = (16,4))
plt.plot(yhat_out[0])
plt.plot(test_data_out)

In [ ]:
#Forecast Out Sample  
x_input_in = test_data_in
x_input_in = x_input_in.reshape((1, n_steps_in))
yhat_in = model_in.predict(x_input_in, verbose=0)
plt.figure(figsize = (16,4))
plt.plot(yhat_in[0])

x_input_out = test_data_out
x_input_out = x_input_out.reshape((1, n_steps_in))
yhat_out = model_out.predict(x_input_out, verbose=0)
plt.figure(figsize = (16,4))
plt.plot(yhat_out[0])

In [ ]:
pred_kas_ech_in = yhat_in[0]
pred_kas_ech_out = yhat_out[0]
train_cut= train[['cash_in_echannel', 'cash_out_echannel','kas_echannel']]
df_pred = pd.DataFrame({'cash_in_echannel':pred_kas_ech_in,'cash_out_echannel':pred_kas_ech_out,'kas_echannel':0})
train_cut = train_cut.append(df_pred).reset_index(drop=True)

In [ ]:
for i in range(1,len(train_cut)):
    train_cut.kas_echannel.iloc[i] = cash_echannel(train_cut.kas_echannel.iloc[i-1],train_cut.cash_in_echannel.iloc[i],train_cut.cash_out_echannel.iloc[i])

In [ ]:
subs_office = y_pred_reg.reshape(31)
subs_ech = train_cut.kas_echannel.iloc[-31:].values

In [ ]:
submission['value'] = np.append(subs_office,subs_ech)
submission.to_csv('subs_03.csv',index=False)